<a href="https://colab.research.google.com/github/Zurehma/AML_Project/blob/zurehma/LAMB_2048.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-optimizer

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import CIFAR10, CIFAR100
import torchvision.models as models
from torch.utils.data import DataLoader

from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import pickle
import numpy as np
import random

# ─── Import LAMB from torch-optimizer ───────────────────────────────────────
from torch_optimizer import Lamb
# ───────────────────────────────────────────────────────────────────────────────

import math

print("PyTorch version:", torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Seeding for reproducibility
seed = 2025
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# transforms
train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ]
)


# load data
def load_data(dataset_name="cifar100", batch_size=128, num_workers=2):
    """Enhanced data loading with stronger augmentation for CIFAR-100"""

    train_transform = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ]
    )
    test_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010)),
        ]
    )

    if dataset_name.lower() == "cifar100":
        train_dataset = CIFAR100(
            root="./data", train=True, download=True, transform=train_transform
        )
        test_dataset = CIFAR100(
            root="./data", train=False, download=True, transform=test_transform
        )
        num_classes = 100
    else:
        raise ValueError(f"Dataset {dataset_name} not supported in enhanced version")

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )
    return train_loader, test_loader, num_classes


# model - LeNet5 from the paper [11]
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        # Input: 3x32x32 (CIFAR has 3 color channels)
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)  # Output: 6x28x28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 6x14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)  # Output: 16x10x10
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 16x5x5

        # fc layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(ImprovedCNN, self).__init__()
        # First conv block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(64)  # using BatchNorm for better training stability
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second conv block
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 384)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(384, 192)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(192, num_classes)

    def forward(self, x):
        # First conv block
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Second conv block
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Flatten and FC
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


def create_model(model_name, num_classes=100):
    if model_name.lower() == "improved_cnn":
        return ImprovedCNN(num_classes=num_classes)
    elif model_name.lower() == "lenet5":
        return LeNet5(num_classes=num_classes)
    else:
        raise ValueError(f"Unknown model: {model_name}")


# train
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc="Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        pbar.set_postfix({"loss": running_loss / total,
                          "acc": 100.0 * correct / total})

    train_loss = running_loss / total
    train_acc = 100.0 * correct / total
    return train_loss, train_acc


# eval
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / total
    test_acc = 100.0 * correct / total
    return test_loss, test_acc


# train config
def train_centralized_model(
    model_name="improved_cnn",
    dataset_name="cifar100",
    batch_size=2048,
    epochs=100,
    lr=0.01,
    momentum=0.9,
    weight_decay=4e-4,
    lr_scheduler="lamb",
    save_path="improved_cnn_model.pth",
):
    # Load data
    train_loader, test_loader, num_classes = load_data(
        dataset_name=dataset_name, batch_size=batch_size
    )

    # Create model
    model = create_model(model_name, num_classes=num_classes).to(device)

    # Print total parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

    # Loss
    criterion = nn.CrossEntropyLoss()

    # Linear‐scaling rule for LR
    scaled_lr = lr * (batch_size / 512)

    # Separate parameters into decay vs. no‐decay groups
    decay_params = []
    no_decay_params = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if param.ndim == 1:
            no_decay_params.append(param)
        else:
            decay_params.append(param)

    # ─── Instantiate LAMB ─────────────────────────────────────────────────────────
    #
    # We set global weight_decay=0.0 because we're handling it per‐group.
    #
    optimizer = Lamb(
        [
            {"params": decay_params,    "weight_decay": weight_decay},
            {"params": no_decay_params, "weight_decay": 0.0}
        ],
        lr=scaled_lr,
        betas=(0.9, 0.999),
        eps=1e-6,
        weight_decay=0.0  # global default; per-group overrides actual decay
    )
    # ────────────────────────────────────────────────────────────────────────────────

    # Learning‐rate scheduler: warmup + cosine, same as LARS code
    if lr_scheduler == "lamb":
        warmup_epochs = 10

        def lr_lambda(epoch):
            if epoch < warmup_epochs:
                return float(epoch + 1) / float(warmup_epochs)
            return 0.5 * (
                1.0
                + math.cos(
                    math.pi * (epoch - warmup_epochs) / (epochs - warmup_epochs)
                )
            )

        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    elif lr_scheduler == "step":
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=[int(epochs * 0.5), int(epochs * 0.75)],
            gamma=0.1
        )
    else:
        raise ValueError(f"Unknown scheduler: {lr_scheduler}")

    # Training history
    history = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
        "lr": [],
    }

    best_acc = 0.0

    # Training loop
    for epoch in range(epochs):
        current_lr = optimizer.param_groups[0]["lr"]
        print(f"\nEpoch [{epoch + 1}/{epochs}] - Learning Rate: {current_lr:.6f}")

        # Train for one epoch
        start_time = time.time()
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        train_time = time.time() - start_time

        # Evaluate
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        scheduler.step()

        print(
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
            f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%, "
            f"Time: {train_time:.2f}s"
        )

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "test_acc": test_acc,
                },
                save_path,
            )
            print(f"New best model saved with accuracy: {test_acc:.2f}%")

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_loss"].append(test_loss)
        history["test_acc"].append(test_acc)
        history["lr"].append(current_lr)

    # Load best model
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    print(f"Best model loaded with accuracy: {checkpoint['test_acc']:.2f}%")

    return model, history


# visualization of model performance
def plot_training_history(history):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    ax1.plot(history["train_loss"], label="Train Loss")
    ax1.plot(history["test_loss"], label="Test Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.legend()
    ax1.set_title("Loss Curves")
    ax1.grid(True)

    ax2.plot(history["train_acc"], label="Train Accuracy")
    ax2.plot(history["test_acc"], label="Test Accuracy")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy (%)")
    ax2.legend()
    ax2.set_title("Accuracy Curves")
    ax2.grid(True)

    ax3.plot(history["lr"])
    ax3.set_xlabel("Epoch")
    ax3.set_ylabel("Learning Rate")
    ax3.set_title("Learning Rate Schedule")
    ax3.set_yscale("log")
    ax3.grid(True)

    plt.tight_layout()
    plt.show()


# run benchmark
def run_centralized_benchmark():
    config = {
        "model_name": "improved_cnn",
        "dataset_name": "cifar100",
        "batch_size": 2048,
        "epochs": 100,
        "lr": 0.01,
        "momentum": 0.9,
        "weight_decay": 4e-4,
        "lr_scheduler": "lamb",
        "save_path": "improved_cnn_model_lamb.pth",
    }

    print(
        f"Running LAMB benchmark with {config['model_name']} on {config['dataset_name']}..."
    )

    model, history = train_centralized_model(**config)
    plot_training_history(history)

    with open("history_lamb.pkl", "wb") as f:
        pickle.dump(history, f)

    print("LAMB benchmark completed!")
    return model, history


def main():
    model, history = run_centralized_benchmark()
    return model, history


if __name__ == "__main__":
    main()
